# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

Given data (features) for a new student (that is not in the current dataset), our goal is to come up with a model that can accurately predict whether the given new student will fail to graduate or not. In this context, it is a classification problem since we are attempting to do a binary classification here i.e., whether a student passes or fails.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# Calculate number of students
n_students = student_data.shape[0]

# Calculate number of features
n_features = student_data.columns[:-1].shape[0]

# Calculate passing students
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]

# Calculate failing students
n_failed = student_data[student_data['passed'] == 'no'].shape[0]

# Calculate graduation rate
grad_rate = (float(n_passed)/(n_students))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(
  X_all, y_all, test_size=num_test, random_state=42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

The three models that can be appropriate for this task are: Random Forest (Ensemble Method), Support Vector Machines (SVM) and Gaussian Naive Bayes (GaussainNB). Given we are faced with a classification task using the dataset, the three models mentioned above are well suited for such classification tasks. 

**Random Forest**: Random Forest classifier is a machine-learning algorithm falling under the category of ensemble learning, which takes the bagging (bootstrap aggergating) approach. Briefly, random forest, as the name suggests, consists of many (random) trees. Each tree is a classifier that is trained independently of other trees. As a classifier, the tree is developed by randomly selecting a subset of features and performing a split over a number of nodes. The result is the predictions provided by the leaf nodes of the tree. Random forest does this for multiple trees. The final predictions are obtained by averaging over the forest. One suggested real world application of Random Forest in machine fault diagnosis. See paper titled: [Application of Random Forest Algorithm in Machine Fault Diagnosis](http://link.springer.com/chapter/10.1007%2F978-1-84628-814-2_82#page-1). Advantages of Random Forests:

1. Highly robust classifier shown to give very accurate results.
2. Capable of handling thousands of features.

Disadvantages include:

1. Training on large datasets can be slow.
2. Can sometimes overfit the data especially if it is noisy.


**Support Vector Machines**: Support Vector Machines (SVMs) are class of supervised learning algorithms that have historically been used for classification purposes. The idea has been to come up with hyper-plane or set of hyper-planes that can separate the samples in two or more classes. They have been for text classification purposes. See paper titled: [Text Categorization with Support Vector Machines: Learning with Many Relevant Features](http://www.cs.cornell.edu/people/tj/publications/joachims_98a.pdf). Advantages of using SVM:

1. Well developed for binary classification task.
2. Work well when we have clear margin of separation.
3. Being effective in high-dimensional spaces.

Disadvantages include:

1. Tuning of hyper-parameters since it has a number of hyper-parameters to tune.
2. Perfomance issues especially with large data sets because the required training time is higher.


**Gaussian Naive Bayes**: Gaussian Naive Bayes is simple yet powerful model. One real life application of this model is in text classification. See the paper titled: [Naive Bayes for Text Classification with Unbalanced Classes](http://www.cs.waikato.ac.nz/~eibe/pubs/FrankAndBouckaertPKDD06new.pdf). Advantages of Gaussain Naive Bayes:

1. Simplicity of implementation
2. No need for hyper-parameter tuning.
3. Good for a dataset with unbalanced classes such as this one i.e., number of students passing is 265 whereas number of students failing is 130.

Disadvantages include:

1. Unrealistic assumption about independence of features.
2. Assumption that the data is normally distributed.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [26]:
"""Slightly changing all three functions in order to report results easily later on """

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    #print "Trained model in {:.4f} seconds".format(end - start)
    return end - start
    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    #print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes'), end - start


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)[0])
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)[0])

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [32]:
# Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Initialize the three models
clf_A = RandomForestClassifier(random_state=0)
clf_B = SVC(random_state=0)
clf_C = GaussianNB()

# Set up the training set sizes
# Splitting this way based on feedback provided by reviewer, @viadanna
X_train_100 = X_train.head(n=100) #train_100[0]
y_train_100 = y_train.head(n=100) #train_100[2]

X_train_200 = X_train.head(n=200) #train_200[0]
y_train_200 = y_train.head(n=200) #train_200[2]

X_train_300 = X_train
y_train_300 = y_train

In [29]:
# Execute the 'train_predict' function for each classifier and each training set size
# Slightly changing the approach since recording results in Markdown table from the output of train_predict

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

all_models = [clf_A,clf_B,clf_C]
all_data = [[X_train_100,y_train_100],[X_train_200,y_train_200],[X_train_300,y_train_300]]

store_results = { 
        'Classifier': [],
        'Training Set Size': [], 
        'Training Time': [], 
        'Prediction Time (test)': [], 
        'F1 Score (train)': [], 
        'F1 Score (test)': []
    }

for clf in all_models:
    for data in all_data:
        X_train_n, y_train_n = data
        time_train = train_classifier(clf, X_train_n, y_train_n)
        f1_train, time_predict_train = predict_labels(clf, X_train_n, y_train_n)
        f1_test, time_predict_test = predict_labels(clf, X_test,y_test)
        store_results['Classifier'].append(clf.__class__.__name__)
        store_results['Training Set Size'].append(X_train_n.shape[0])
        store_results['Training Time'].append("{:.4f}".format(time_train))
        store_results['Prediction Time (test)'].append("{:.4f}".format(time_predict_test))
        store_results['F1 Score (train)'].append(f1_train)
        store_results['F1 Score (test)'].append(f1_test)

        
final = pd.DataFrame(store_results)
final[['Classifier','Training Set Size', 'Training Time', 'Prediction Time (test)', 
       'F1 Score (train)', 'F1 Score (test)']]

#for clf in all_models:
#    for data in all_data:
#        train_predict(clf, data[0], data[1], X_test, y_test)

,Classifier,Training Set Size,Training Time,Prediction Time (test),F1 Score (train),F1 Score (test)
0,RandomForestClassifier,100,0.0493,0.0065,0.992248,0.661417
1,RandomForestClassifier,200,0.0472,0.0058,0.996364,0.753623
2,RandomForestClassifier,300,0.0384,0.0057,0.992701,0.740741
3,SVC,100,0.0012,0.0007,0.877698,0.774648
4,SVC,200,0.0028,0.0010,0.867925,0.781457
5,SVC,300,0.0057,0.0025,0.876068,0.783784
6,GaussianNB,100,0.0012,0.0004,0.846715,0.802920
7,GaussianNB,200,0.0011,0.0004,0.840580,0.724409
8,GaussianNB,300,0.0012,0.0004,0.803783,0.763359


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Random Forest**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 |	0.0482 |	0.0062 |	0.992248 	|0.661417
| 	200 |	0.0420 |	0.0062 |	0.996364 |	0.753623
| 	300 |	0.0425 |	0.0058 |	0.992701 |	0.740741

** Classifer 2 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 |	0.0012 |	0.0007 |	0.877698 |	0.774648|
| 		200 |	0.0030 |	0.0010 |	0.867925| 	0.781457|
| 	 	300 |	0.0064 |	0.0019 |	0.876068 |	0.783784 |

** Classifer 3 - Gaussian Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100 |	0.0009 |	0.0003 	|0.846715 |	0.802920|
| 	200 |	0.0009 |	0.0003 |	0.840580 |	0.724409|
| 	300 |	0.0010 |	0.0003 |	0.803783 |	0.763359  |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

In the the context of limited resources, cost and performance, Gaussian Naive Bayes appears to be the best model to use. This true for a number of reasons:

1. It has the lowest training and prediction time compared to Random Forest and Support Vector Machine across all training set sizes.
2. Unlike Random Forest and Support Vector Machines, the Gaussian Naive Bayes does not have any significant number of hyper-parameters to tune. In the sklearn implementation, it only has one hyper-parameter, the [priors](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html), which can be set to none in most cases. This further saves time as there is no need to perform GridSearch.
3. The average F<sub>1</sub> score of Gaussian Naive Bayes (across all 3 different training sizes) is 0.76. It is better than the average F<sub>1</sub> score of Random Forest but only slightly lower than the average F<sub>1</sub> score of SVM (by 0.02). Since the difference is so marginal, we can sufficiently rely on Gaussian Naive Bayes.

In [20]:
## Averaging the F1 scores across different training set size
rf_f1_avg = np.mean([0.661417,0.753623,0.740741])
svm_f1_avg = np.mean([0.774648,0.781457,0.783784])
gnb_f1_avg = np.mean([0.802920,0.724409,0.763359])

"RF: {0:.2f}, SVM: {1:.2f}, GNB: {2:.2f}".format(rf_f1_avg,svm_f1_avg,gnb_f1_avg)

'RF: 0.72, SVM: 0.78, GNB: 0.76'

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Gaussian Naive Bayes is a fairly simple yet highly effective machine learning model. It has shown to do well in its ability to predict which students will pass and which students will fail. Data about the students and their pass/fail status is first split into two parts. One part is called the training set, on which the model is trained. The other set is called testing set, on which the trained model is evaluated.

Given a set of features for a given student such as student's school, sex, age, study time, etc, the model uses a statisical method called the Bayes Theorem to determine the probability of the student failing or passing. It's called Gaussian because it assumes the data to be follow a Gaussian distribution and it's naive because it assumes the features in the data to be statistically independent.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

**Since the chosen model, Naive Gaussian Bayes, does not have any parameters to tune, I chose the next best model, SVM, for tuning purpose**.

In [24]:
# Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV


# Create the parameters list you wish to tune
parameters = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear'], 'random_state':[0]},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 
  'kernel': ['rbf'], 'random_state':[0]},
 ]

# TODO: Initialize the classifier
clf = SVC(random_state=0,probability=True)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf,param_grid=parameters,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)[0])
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)[0])

Tuned model has a training F1 score of 0.8323.
Tuned model has a testing F1 score of 0.7945.


In [25]:
clf

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=True, random_state=0, shrinking=True, tol=0.001,
  verbose=False)

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final model's F1 score for training is 0.8323 and for testing is 0.7945. The F1 score for untuned model's training was 0.876068 and testing was 0.783784.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.